In [ ]:
#Longhand takes a handwritten document corpus and returns an immersive visualization.

In [ ]:
#Pre-processing for image OCR
#Portions from Gabe Pizzorno 'Workflow-clean' script
import cv2
import numpy as np
import os
import sys
import subprocess
import re

#declarations
path='xxx' #target image directory
out='xxx' #transcription output path
listFiles=os.listdir(path)
imgNo = 0

#file/os operations
for file in sorted(listFiles):
    if not file.startswith('.'):
        print(path +'/'+ file)
        image = cv2.imread(path +'/'+ file)
        fileName=file.split('.')[0]
        img_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_bw = cv2.adaptiveThreshold(img_gs, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 91, 5)
        img_inv = cv2.bitwise_not(img_bw)
        cv2.imshow(fileName, img_inv)
        cv2.imwrite(out+"/"+fileName+".jpg", img_inv) 
        print(fileName + " pre-processed")
        cv2.waitKey(10)
        imgNo = int(imgNo) + 1
    cv2.destroyAllWindows()  

In [ ]:
#Run Handprint from local directory
!handprint --service microsoft,google,amazon-textract --debug - "xxx" --no-grid --extended --output "xxx"


In [ ]:
#Run Handprint using URLs from local .txt file
!handprint --service microsoft,google,amazon-textract --from-file "xxx" --no-grid --extended --output "xxx"

In [ ]:
####Bag-of-Words txt output and string search from HandPrint transcriptions. MNC - 5/21
###Built to work with HandPrint, by Mike Hucka: https://github.com/caltechlibrary/handprint

import os
import sys
import re

#paths
cardPath = 'xxx'#plain text list of urls or filenames (pre-existing)
outPath = 'xxx' #HP output (pre-existing)
textOut = 'xxx'#Bag-of-Words output
searchOut = 'xxx'#user search output
listFiles = os.listdir(outPath)
count = 0

#declarations
headers = [] 
transcriptions = []
BoW = open(textOut, "a")

#user search
search = open(searchOut, "a")
inputString = input("Search document for: ")
inputString = str(inputString)

#generate header list
with open (cardPath, 'rt') as myfile: 
    for line in myfile:
        headers.append(line)

#append headers and transcriptions to bag-of-words 
for file in sorted (listFiles):
    if not file.startswith('.') and file.endswith (".handprint-microsoft.txt"):
    #if not file.startswith('.') and file.endswith (".handprint-amazon-textract.txt"):
    #if not file.startswith('.') and file.endswith (".handprint-google.txt"):
        BoW.write("\n")
        BoW.write(headers[count]) #write card location to disk
        #print("\n")
        print((headers[count]) + "added")
        #contents = open(outPath + "/document-" + str(count + 1) + ".handprint-microsoft.txt", "r") #read card transcription
        contents = open(outPath + "/" + file, "r") #read card transcription
        transcriptions.append(contents.read())
        #print(transcriptions[count])
        #print("\n")
        BoW.write(transcriptions[count]) #write transcriptions to disk
        BoW.write("\n")
        count = count + 1
        #print("\n")    
BoW.close()

#search
BoW = open(textOut, "r")
count = 0

while count < len(headers):
    if inputString in transcriptions[count]:
        print(headers[count])
        print(transcriptions[count])
        search.write("\n")
        search.write(headers[count])
        search.write(transcriptions[count])
        search.write("\n")
        print("\n")
        count = count + 1
    elif inputString not in transcriptions[count]:
        #print("\n")
        print("No such string found in transcription " + str(count + 1))
        #print("\n")
        count = count + 1
search.close()

print("\n")
print("have a nice day")
        
###To DO
### more inclusive search
### LocateXT integration


In [ ]:
###pdf parser
###PyMuPDF: https://pymupdf.readthedocs.io/en/latest/

import sys 
import fitz
fname = "xxx"  # get document filename
doc = fitz.open(fname)  # open document
out = open(fname + ".txt", "wb")  # open text output
for page in doc:  # iterate the document pages
    text = page.get_text().encode("utf8")  # get plain text (is in UTF-8)
    out.write(text)  # write text of page
    out.write(bytes((12,)))  # write page delimiter (form feed 0x0C)
out.close()

In [ ]:
###Natural Language Processing
##Modified from "named-entity-recognition" repo by Mary Chester-Kadwell
##(https://github.com/mchesterkadwell/named-entity-recognition/blob/main/LICENSE)

import spacy
import en_core_web_sm
from spacy import displacy
from pathlib import Path
import matplotlib.pyplot as plt
from collections import Counter

#declarations
nlp = en_core_web_sm.load()
text_file = Path('data', 'xxx')
body_parts = open("xxx", "r")

# Open the file, read it and store the text with the name `iliad`
with open(text_file, encoding="utf-8") as file:
    iliad = file.read()

#pass text to SpaCy language model()
document = nlp(iliad)
document.text

#generate list of body common parts
#Note:"back" removed
list_of_parts = []
for line in body_parts:
    stripped_line = line.strip()
    #line_list = stripped_line.split()
    list_of_parts.append(stripped_line)
#print(list_of_parts)

#identify nouns in document
nouns = []
pos_tags = [(token.text, token.pos_, token.tag_) for token in document if token.text.isalpha()]
for token in document:
    if token.text in list_of_parts:
        nouns.append(token.text)
#print(nouns)

persons = []
for entity in document.ents:
    if entity.label_ == "PRODUCT":
        persons.append(entity.text)
        #print(f'{entity.text}: {entity.label_}')
        
#print high-frenquency nouns
word_freq = Counter(nouns)
common_words = word_freq.most_common(25)
print(common_words)

# Display the plot inline in the notebook with interactive controls
%matplotlib notebook

# Get a list of the most common words
words = [word for word,_ in common_words]

# Get a list of the frequency counts for these words
freqs = [count for _,count in common_words]

# Set titles, labels, ticks and gridlines
plt.title("Body part mentions in ""Women in Chains")
plt.xlabel("Word")
plt.ylabel("# of appearances")
plt.xticks(range(len(words)), [str(s) for s in words], rotation=90)
plt.grid(b=True, which='major', color='#333333', linestyle='--', alpha=0.5)
plt.gcf().subplots_adjust(bottom=0.35)

# Plot the frequency counts
plt.plot(freqs)

# Show the plot
plt.show()
plt.savefig('xxx', bbox_inches="tight")

#display named entities
displacy.render(document, style="ent")

body_parts.close()
file.close()


In [ ]:
###Topic Modeling (LDA) Longhand inputs
###Cook 2021
##portions of code from https://radimrehurek.com/gensim/auto_examples
import os
import sys
import gensim
from gensim import corpora
import pprint
from collections import defaultdict
from gensim import models
from gensim.models import LdaModel
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#declarations
doc = open("xxx", "r")
doc = doc.read()
documents = [doc]

stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in doc.lower().split() if word not in stoplist]
    for doc in documents
]
#print(texts)

#create dictionary
dictionary = corpora.Dictionary(texts)
print(dictionary)

#create corpus
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

doc.close()